In [1]:
!pip install robobrowser

In [2]:
# Python 2-3 compatibility
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from builtins import *

In [3]:
import requests
from getpass import getpass
from time import sleep
import json

In [4]:
import tinder_api as api
import phone_auth

In [23]:
print('Enter your phone number for the Tinder account, in international format. '
      'Be sure that you have already set up the account and verified the number and email on Tinder!')

phone_num = input("Phone Number: ")

req_code = phone_auth.sendCode(phone_num)
if req_code is False:
    raise ValueError('There was an error when sending the phone number!')
else:
    print('Your request code is:', req_code)

sms_tries = 0
sms_success = False
for sms_tries in range(3):
        
    sms_code = input("Please enter the code you've received by sms: ")
    tinder_token = phone_auth.getToken(phone_num, sms_code, req_code)
    if tinder_token is not None:
        sms_success = True
        api.headers.update({"X-Auth-Token": tinder_token})
        break

if not sms_success:
    raise ValueError("SMS token incorrectly entered 3 times, run the cell again")
    
print("Tinder token:", tinder_token)

Enter your phone number for the Tinder account, in international format. Be sure that you have already set up the account and verified the number and email on Tinder!


/home/ryan/.anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://graph.accountkit.com/v1.2/start_login?access_token=AA%7C464891386855067%7Cd1891abb4b0bcdfa0580d9b839f4a522&credentials_type=phone_number&fb_app_events_enabled=1&fields=privacy_policy%2Cterms_of_service&locale=en_US&phone_number=16072695182&response_type=token&sdk=ios
Your request code is: 0a02c6bc08c48dfa083f4348c20fb0ad
Please enter the code you've received by sms: 805811


/home/ryan/.anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/ryan/.anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


NameError: name 'headers' is not defined

In [14]:
def get_name(recs_result):
    """Helper function to take ain item of the recommended users and convert them to a name."""
    return recs_result['user']['name'].lower()

In [15]:
def find_people(names, num_to_search, delay=0.5):
    """Find people on tinder by name
    
    Args:
        names: A list of strings to search for
        num_to_search: The minimum number of people to search through
        delay: The time in seconds between each request to tinder's servers.
        
    Returns: A tuple with a list of python dicts for each user that matched a name in `names`,
        and the number of people searched through.
    """
    if isinstance(names, str):
        raise ValueError("`names` should be an iterable!")
    names = set(n.lower() for n in names)
    count = 0
    results = []
    while count < num_to_search:
        try:
            response = requests.get(api.host+'/v2/recs/core?locale=en-US', headers=api.headers)
            if not response.ok:
                break
            print(response.json())
            recs = response.json()["data"]["results"]

            for rec in recs:
                rec_name = get_name(rec)
                if rec_name in names:
                    results.append(rec)
            count += len(recs)
            sleep(delay)
        except Exception as e:
            print(e)
        
    return results, count

In [16]:
def get_link_from_id(id):
    """Gets the sharable link for someone's profile. To view you must have the app. 
    Limited uses for each generated link."""
    response = requests.post(api.host + '/user/%s/share' % id, headers=api.headers)
    if not response.ok:
        return None
    return response.json()['link']

In [21]:
def set_loc(lat, lon):
    """Returns whether the location was updated was successfully."""
    data = {'lat':lat, 'lon':lon}
    response = requests.post(api.host + '/user/ping', headers=api.headers, data=json.dumps(data))
    return response.ok

In [25]:
set_loc(42.4426977,-76.5333944)  # NYC

True

In [11]:
# this is easier to do in the app, but in case you want to do it from python, here it is
api.change_preferences(age_filter_max=21, age_filter_min=22, distance_filter=2)

{'meta': {'status': 400}, 'error': {'code': 41002}}

In [12]:
results, num_searched = find_people(['sarah'], 1000)
results, num_searched

([], 0)

In [13]:
# extract the user ids and sharable links from the results for future reference and easier viewing
for r in results:
    _id = r['user']['_id']
    print('id:', _id, 'link:', get_link_from_id(_id))